<a href="https://colab.research.google.com/github/DonRoboto/KalmanFilter_v1/blob/main/SimulatedKalmanFilter_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# import math functions
from math import *
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
#kalman filter 1d
class Kalman_1D():

  def __init__(self, x, y):
    self.x = x
    self.y = y


  def fitness(self):
    x = self.x
    y = self.y

    #Sphere
    #return x**2.0 + y**2

    #Schwefel
    #return (418.9829 * 2) -  ( (x * np.sin(np.sqrt(np.abs(x))) ) +  (y * np.sin(np.sqrt(np.abs(y))) ) )

    #Rastrigin 
    #return (10*2) + ( (x**2 - (10*np.cos(2*np.pi*x))) + (y**2 - (10*np.cos(2*np.pi*y))))

    #Ackley
    return -20 * np.exp(-0.2*np.sqrt(0.5*(x**2+y**2)) ) - exp(0.5*(np.cos(2*np.pi*x)+np.cos(2*np.pi*y))) + np.e + 20

    
  
  def measure(self, x_true, y_true):
    rnd_x = random.uniform(0, 1)
    rnd_y = random.uniform(0, 1)

    z_x = self.x + np.sin(rnd_x * 2 * np.pi) + (np.abs(self.x - x_true))
    z_y = self.y + np.sin(rnd_y * 2 * np.pi) + (np.abs(self.y - y_true))
    return z_x, z_y

  def estimate(self, z_x, z_y, K):    
    x_est = self.x + K * (z_x -  self.x )
    y_est = self.y + K * (z_y -  self.y )
    return x_est, y_est

In [3]:
class Poblacion():
  p = []

  def __init__(self, n, x_min, x_max):

    for i in range(n):
      x_1 = random.uniform(x_min, x_max)
      y_1 = random.uniform(x_min, x_max)
      ind = Kalman_1D(x_1, y_1)
      self.p.append(ind)

In [4]:
v_min = -500
v_max = 500
n_pob = 50
pob = Poblacion(n_pob, v_min, v_max)

In [5]:

f_best = 1000000
f_true = 1000000

x_best = 1000000
y_best = 1000000

x_true = 1000000
y_true = 1000000

for i in range(n_pob):
  
  if pob.p[i].fitness() < f_best:
    x_best = pob.p[i].x
    y_best = pob.p[i].y
    f_best = pob.p[i].fitness()

  if  f_best<f_true:
    x_true = x_best
    y_true = y_best
    f_true = f_best


print("x_true: {}".format(x_true))
print("y_true: {}".format(y_true))
print("f_true: {}".format(f_true))

x_true: 416.036311514551
y_true: 146.0132631923476
f_true: 20.039645478991865


In [6]:

P = 1000
Q = 0.5
R = 0.5


for t in range(100):
   
  P = P + Q
  err = []

  for j in range(n_pob):

    z_x, z_y = pob.p[j].measure(x_true, y_true) 
    
    #kalman gain
    K = P/(P + R)
   
    x_e, y_e = pob.p[j].estimate(z_x, z_y, K)

    if abs(x_e)>v_max:
      x_e = random.uniform(v_min, v_max)
    if abs(y_e)>v_max:
      y_e = random.uniform(v_min, v_max)

    pob.p[j].x = x_e
    pob.p[j].y = y_e

    err.append(np.abs(f_true - pob.p[j].fitness()))

    #fitness evaluation
    if pob.p[j].fitness() < f_best:
      x_best = pob.p[j].x
      y_best = pob.p[j].y
      f_best = pob.p[j].fitness()


    if  f_best<f_true:
      f_true = f_best
      x_true = x_best
      y_true = y_best

  P = np.cov(err)
  P = (1 - K) * P


print("\n")
print("x_true: {}".format(x_true))
print("y_true: {}".format(y_true))
print("f_true: {}".format(f_true))



x_true: -0.012611487879711891
y_true: 0.08445800309087859
f_true: 0.42469742363402574
